In [ ]:
#ingest data
import pandas as pd
import pickle
from sklearn.linear_model import Lasso



#For Evaluating the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Algorithm used - LinearRegression
from sklearn.linear_model import LinearRegression

#To convert million , billion string into float
import re

#For plotting
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer

import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("youtube-prediction-experiment")

In [ ]:
# Load the data
def read_dataframe(filename):
    data = pd.read_csv(filename)

    # Drop rows with NaN values
    data.dropna(inplace=True)
    
    data['VIEWS'] = clean_numeric_column(data['VIEWS'])
    data['TOTAL_NUMBER_OF_VIDEOS'] = clean_numeric_column(data['TOTAL_NUMBER_OF_VIDEOS'])
    data['SUBSCRIBERS'] = clean_numeric_column(data['SUBSCRIBERS'])
    
    print(data)
    
    
    return data


    
def clean_numeric_column(column):
    cleaned_column = []
    for value in column:
        if isinstance(value, str):
            if 'K' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000)
            elif 'M' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000)
            elif 'B' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000000)
            else:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)))
        else:
            cleaned_column.append(value)
    return cleaned_column

In [ ]:
df_train = read_dataframe('/workspaces/mlops_zoomcamp_project/dataset/youtube_test.csv')
df_val = read_dataframe('/workspaces/mlops_zoomcamp_project/dataset/youtube_train.csv')

In [ ]:
print(len(df_train))
print(len(df_val))

In [ ]:
# Define features and target
numeric_features = ['VIEWS', 'TOTAL_NUMBER_OF_VIDEOS']
categorical_features = ['CATEGORY']

# Prepare data for training
train_dicts = df_train[categorical_features + numeric_features].to_dict(orient='records')

# Prepare data for training
val_dicts = df_val[categorical_features + numeric_features].to_dict(orient='records')



# Initialize DictVectorizer
dv = DictVectorizer()


X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

print("X_train shape:", X_train.shape)

# Extract target variable
y_train = df_train['SUBSCRIBERS'].values
y_val = df_val['SUBSCRIBERS'].values
print("y_train shape:", y_train.shape)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
# Predict the number of subscribers for each channel in the dataset
df_val['Predicted Subscribers'] = lr.predict(X_val)

# Save the predictions to a new CSV file
df_val.to_csv('/workspaces/mlops_zoomcamp_project/dataset/Result_youtube_channels_with_predictions.csv', index=False)
print("Predictions saved to 'youtube_channels_with_predictions.csv'")

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Anamika")

    mlflow.log_param("train-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_train.csv")
    mlflow.log_param("valid-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_test.csv")
    

    alpha = 0.2
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/workspaces/mlops_zoomcamp_project/models/lin_reg.bin", artifact_path="models_pickle")

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=2
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)